In [3]:
import requests
import pandas as pd
import numpy as np
import sqlite3
import time
import Levenshtein

from IPython.display import clear_output

In [8]:
def process_game_list_json(jsn_rslt:str) -> pd.DataFrame:
    ## !!!!!!!! Transform 1 : parseing out the json data into a usable DataFrame !!!!!!!!!!
    ## !!!!!!!! Transform 2 : Replacing the Headers !!!!!!!!
    return pd.DataFrame([(x['id'],x['names']['international'] )for x in jsn_rslt['data']], columns=['id','name'])

def create_game_list() -> None:
    '''
    pulls the games list from the speedruns.com api. 
    uses increments of 1000, and the bulk api call
    will write the output to the database for each group of 1000
    '''
    inc = 1000
    off_set = 0

    url = 'https://www.speedrun.com/api/v1/games'
    db = sqlite3.connect("game_data.db")
    cur = db.cursor()

    cur.execute('DROP TABLE IF EXISTS sp_games_stage')

    
    while True:
        payload = {'_bulk':1,
              'max':inc,
              'offset':off_set,}
        
        rspns = requests.get(url,payload)
        jsn_data= rspns.json()
        game_data_df = process_game_list_json(jsn_data)
        game_data_df.to_sql('sp_games_stage', db,if_exists='append')

        if len(jsn_data['data']) == 1000:
            off_set += inc
        else:
            db.close()
            break
            
        time.sleep(3)
        print(f'getting upto {off_set}')

def find_mathcing_id(chck_val:str, chck_series: pd.DataFrame) -> str:
    #### transform 3 Fuzzy Matching
    '''
    review a string pass to another dataframe
    and if a match is found return the id column
    dataframe layout should be [id, name]
    '''
    ## grab the first letter of search value to shorten the list
    frst_val = chck_val[0]

    ## shorten list by frist val varable
    review_df = chck_series[chck_series['name'].str.startswith(frst_val)]
    print(f'length of review list {len(review_df)}')

    ## Create an empty list to hold all possible match 
    match_list = list()
    for index, row in review_df.iterrows():
        
        ## get the jaro_winkler % and if above .95 then add to the list
        dist_val = Levenshtein.jaro_winkler(chck_val,row.iloc[2])
        if dist_val >= .95:
            match_list.append((dist_val,chck_val,row.iloc[2],row.iloc[1]))

    
    ## return the best matching value from the list
    if len(match_list) > 0:
        sorted_list = sorted(match_list, key=lambda x: x[0], reverse = True)
        print(f'Match Found!! check: {chck_val} | match: {sorted_list}')            
        return sorted_list[0][3]


def process_game_results(json_txt:str) -> pd.DataFrame:

    appnd_list = list()

    for i in json_txt['data']:
        parse_data = [i['game'], i['category']['data']['id'],i['category']['data']['name'],i['category']['data']['rules'],]
        
        rns = parse_runs_list(i['runs'])
        if rns != None:
            for rn in rns:
                parse_data.append(rn)

        appnd_list.append(parse_data)

    return pd.DataFrame(appnd_list,columns=['spd_rn_id','category_id','category_name','category_rules','run_1_time','run_2_time','run_3_time'])

def parse_runs_list(rund:dict) -> list:
    match len(rund):
        case 1:
            return [rund[0]['run']['times']['primary_t'],np.nan,np.nan]
        case 2:
            return [rund[0]['run']['times']['primary_t'],rund[1]['run']['times']['primary_t'],np.nan]
        case 3:
             return [rund[0]['run']['times']['primary_t'],rund[1]['run']['times']['primary_t'],rund[2]['run']['times']['primary_t']]
        case _:
            return [np.nan,np.nan,np.nan]

def get_next_url(json_data:str) -> str:


    if len(json_data['pagination']['links']) > 0:
        if json_data['pagination']['links'][0]['rel'] == 'next':
            return json_data['pagination']['links'][0]['uri']

    return None
    
def fetch_game_records(game_list: pd.Series):

    db = sqlite3.connect("game_data.db")
    cur = db.cursor()
    cur.execute('DROP TABLE IF EXISTS spd_rns_data')
    
    for indx,gameid in game_list.items():
        clear_output(wait = True)
        url = f'https://www.speedrun.com/api/v1/games/{gameid}/records?embed=category%2Cplatform&offset=0'
        print(indx)

        while True:
            rslts = requests.get(url)

            if rslts.status_code == 200:
                print(f'got a hit for {indx}')
                json_data = rslts.json()
                df = process_game_results(json_data)
                df.to_sql('spd_rns_data',db, if_exists='append')
                
                url = get_next_url(json_data)
                if url == None:
                    print('No next Record')
                    break
            else:
                print(f'Bad http code {rslts.status_code}')
                break
                
        time.sleep(.5)

    print('broke out of loop')
    con.close()

                

# Transform 1 & 2
occures in the below funtion via process_game_list_json function.
1. parsing the JSON file to a usable format
2. adding heads to the dataframe

In [9]:
### retive the list of games from speed runs API
create_game_list()

getting upto 1000
getting upto 2000
getting upto 3000
getting upto 4000
getting upto 5000
getting upto 6000
getting upto 7000
getting upto 8000
getting upto 9000
getting upto 10000
getting upto 11000
getting upto 12000
getting upto 13000
getting upto 14000
getting upto 15000
getting upto 16000
getting upto 17000
getting upto 18000
getting upto 19000
getting upto 20000
getting upto 21000
getting upto 22000
getting upto 23000
getting upto 24000
getting upto 25000
getting upto 26000
getting upto 27000
getting upto 28000
getting upto 29000
getting upto 30000
getting upto 31000
getting upto 32000
getting upto 33000
getting upto 34000
getting upto 35000
getting upto 36000
getting upto 37000
getting upto 38000
getting upto 39000
getting upto 40000
getting upto 41000
getting upto 42000
getting upto 43000
getting upto 44000


In [145]:
with sqlite3.connect("game_data.db") as con:
    spd_rn_data = pd.read_sql('select * from sp_games_stage', con)
    gm_lst_data = pd.read_sql("SELECT ID, GAME_NAME FROM ttb_game_list", con)

print(len(spd_rn_data))
print(spd_rn_data.head())

44265
   index        id                                   name
0      0  pdv9v5k1   Alex Kidd 3 - Curse in Miracle World
1      1  v1plx046                           Battle Mania
2      2  o1y32j26                       Bike Unchained 2
3      3  y65rm041        BIRDIE WING -Golf Girls' Story-
4      4  j1n4pkx6           Body by Milk Race and Refuel


# Transform 3 - Fuzzy Matching to match the speedruns.com id to the Playtime ID
the matching is preformed in the function find_mathcing_id

In [149]:
## !!!!!!!! this takes a while to run as it will be doing at least 60k reviews around an hour. maybe look at refactoring
for index, row in gm_lst_data.iterrows():
    clear_output(wait = True)
    
    chck_val = row.iloc[1]
    
    match_id = find_mathcing_id(chck_val,spd_rn_data)

    if match_id != None:
        print(match_id)
        gm_lst_data.loc[index,'spd_rn_id']= match_id
        

length of review list 5671
Match Found!! check: Sinistar | match: [(0.96, 'Sinistar', 'Sinisistar', 'm1mnm04d')]
m1mnm04d


In [5]:
### write the table to the database
with sqlite3.connect("game_data.db") as con:
    gm_lst_data.to_sql('game_list_bridge',con)

NameError: name 'gm_lst_data' is not defined

# Transform 4 droping duplicautes to pull more data

In [7]:
with sqlite3.connect("game_data.db") as con:
    gm_lst_data = pd.read_sql('SELECT * FROM game_list_bridge',con)

gms_to_get_runs = gm_lst_data['spd_rn_id']
gms_to_get_runs = gms_to_get_runs.drop_duplicates()

# Transform 5+ pulling and processing Run data
I'm agian splint up the json file into differnt fields, renaming columns and loading it to the database. Via the function fetch_game_records

In [9]:
### this also takes a long time to run due to the 100 request per min rate limit. I slow the request down by .5 seconds to avoid the list
## interrupted it just for time
fetch_game_records(gms_to_get_runs)

60396
got a hit for 60396
No next Record
broke out of loop


# Print out Final Datasets: Speed Run Data

In [10]:
with sqlite3.connect("game_data.db") as con:
    df_runs = pd.read_sql('select * from spd_rns_data', con)
    df_mstr_game = pd.read_sql('select * from game_list_bridge', con)

df_runs

,index,spd_rn_id,category_id,category_name,category_rules,run_1_time,run_2_time,run_3_time
0,0,369penl1,zd3wp5nk,Any%,,7488.0,7747.0,8303.0
1,1,369penl1,9kvowr02,Any% Co-Op,#### **Completion**\r\n* **Any% Co-Op** - Comp...,9782.0,11859.0,13487.0
2,2,369penl1,xd1v3x8d,100%,#### **Completion**\n\n- **100%** - Obtain Max...,22485.0,28807.0,NaN
3,3,369penl1,jdz4vm3k,100% Co-Op,#### **Completion**\n\n- **100%** - Obtain Max...,27525.0,NaN,NaN
4,4,369penl1,824gxxm2,Geared,,6831.0,8047.0,8095.0
...,...,...,...,...,...,...,...,...
164134,8,o6gn0vx1,02qel8p2,pinball%,Complete the pinball minigame. Timer starts on...,NaN,NaN,NaN
164135,9,o6gn0vx1,824qrmgk,bubble%,Complete the bubble bobble minigame. Timer sta...,NaN,NaN,NaN
164136,10,o6gn0vx1,9d85gj3d,minesweeper%,Complete the minesweeper minigame without losi...,NaN,NaN,NaN
164137,0,m1mnm04d,jdzljjv2,All Bosses,"1.Time begins on""Forest of the Dead Area1"" sel...",1509.0,1540.0,1605.0


# Print Out Final Dataset Master Game Tabel

In [283]:
df_mstr_game

,index,id,game_name,spd_rn_id
0,0,1,Borderlands 3,369penl1
1,1,2,Dying Light,j1lnx46g
2,2,3,Middle-Earth: Shadow of War,lde77j13
3,3,4,Counter-Strike: Global Offensive,ldemjl13
4,4,5,Grand Theft Auto IV,nj1nee1p
...,...,...,...,...
60392,60392,60393,Left-Hand Path,None
60393,60393,60394,Astro Attack,None
60394,60394,60395,Apocalypse: Party's Over,None
60395,60395,60396,10mg :),o6gn0vx1


# Summary

This API for Milestone 4 turns out to be a pretty labor-intensive effort. I had to work around rate limits and access more than one API’s with lots of data. I ended up creating two data sets: A master list of games and ID, which will allow me to mix my data with previous Milestone pulls, and a data set of the top runs by Category for each game. 
Outside the fuzzy matching, the transforms this time are boring; they are mainly just parsing Jason records, renaming columns, etc. I spent a lot of my time reworking the fuzzy matching logic to match two large datasets. 
I also really don’t like some of the code I wrote for this assignment, it all needs a refactor. It works but barely.
Ethically, there is nothing to be concerned about with the data handling here. I’ve just pulled values and matched them to another dataset.
